In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = datasets.load_breast_cancer()
X, y = df.data, df.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [4]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

In [5]:
model = LogisticRegression(n_features)

In [6]:
learning_rate = 0.01

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
num_epochs = 1000
for epoch in range(num_epochs):
    #Forward and Loss
    y_predicted = model.forward(X_train)
    loss = criterion(y_predicted, y_train)
    
    #Backward and Update
    loss.backward()
    optimizer.step()
    
    #Zero
    optimizer.zero_grad()
    
    #Logging
    if (epoch + 1) % 100 == 0:
        print(f'Epoch: {epoch + 1} Loss: {loss:.8f}.')

Epoch: 100 Loss: 0.24216551.
Epoch: 200 Loss: 0.18397513.
Epoch: 300 Loss: 0.15788323.
Epoch: 400 Loss: 0.14225881.
Epoch: 500 Loss: 0.13158654.
Epoch: 600 Loss: 0.12371049.
Epoch: 700 Loss: 0.11759219.
Epoch: 800 Loss: 0.11266286.
Epoch: 900 Loss: 0.10858180.
Epoch: 1000 Loss: 0.10513107.


In [8]:
with torch.no_grad():
    y_test_pred = model.forward(X_test)
    test_loss = criterion(y_test_pred, y_test)
    print(test_loss.item())

0.09011965990066528
